# process 20newsgroup dataset 

We download [20newsgroup dataset](http://qwone.com/~jason/20Newsgroups/20news-18828.tar.gz) and store it at /Users/huangwaleking/git/detector/20news-jupyter。

Firstly, we read it by nltk.

In [2]:
import nltk
newsgroups = \
  nltk.corpus.PlaintextCorpusReader('/Users/huangwaleking/git/detector/20news-18828', '.*/[0-9]+', encoding='latin1')

In [11]:
ids = newsgroups.fileids()
print("在20newsgroups中有%s个文件" % len(ids))
print("前20个文件的id是：\n %s" % ids[:20])
print("第一个文件的内容是%s" % newsgroups.words(fileids=ids[0]))
print("newsgroups.words的类型是%s" % type(newsgroups.words(fileids=ids[0])))

在20newsgroups中有18828个文件
前20个文件的id是：
 ['alt.atheism/49960', 'alt.atheism/51060', 'alt.atheism/51119', 'alt.atheism/51120', 'alt.atheism/51121', 'alt.atheism/51122', 'alt.atheism/51123', 'alt.atheism/51124', 'alt.atheism/51125', 'alt.atheism/51126', 'alt.atheism/51127', 'alt.atheism/51128', 'alt.atheism/51130', 'alt.atheism/51131', 'alt.atheism/51132', 'alt.atheism/51133', 'alt.atheism/51134', 'alt.atheism/51135', 'alt.atheism/51136', 'alt.atheism/51139']
第一个文件的内容是[u'From', u':', u'mathew', u'<', u'mathew', u'@', ...]
newsgroups.words的类型是<class 'nltk.corpus.reader.util.StreamBackedCorpusView'>
